##Imports 

In [ ]:

import pandas as pd
import torch
import urllib.request

Download the pretrained caption aware transformer model, along with cloning the entire repo with requirements file

In [ ]:

model = torch.hub.load('saahiluppal/catr', 'v3', pretrained=True)
!git clone https://github.com/saahiluppal/catr.git
!pip install -r catr/requirements.txt

!mkdir images

/usr/local/lib/python3.7/dist-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  "You are about to download and run code from an untrusted repository. In a future release, this won't "
Downloading: "https://github.com/saahiluppal/catr/zipball/master" to /root/.cache/torch/hub/master.zip
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.

  0%|          | 0.00/171M [00:00<?, ?B/s]

Downloading: "https://github.com/saahiluppal/catr/releases/download/0.2/weight493084032.pth" to /root/.cache/torch/hub/checkpoints/weight493084032.pth


  0%|          | 0.00/322M [00:00<?, ?B/s]

Cloning into 'catr'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 121 (delta 46), reused 36 (delta 36), pack-reused 59
Receiving objects: 100% (121/121), 3.05 MiB | 18.70 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 14.6 MB/s 
     |████████████████████████████████| 6.6 MB 53.7 MB/s 
     |████████████████████████████████| 101 kB 13.8 MB/s 
     |████████████████████████████████| 596 kB 73.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


##Dataset Pre-processing

Create the dataframe, which expects tsv file for each and disaster. The dataset is in \datasets folder. In the end we are dropping columns which are not necessary.

In [ ]:
# create the dataframe. It expects tsv file for each disaster


#df = pd.read_csv('srilanka_floods_final_data.tsv', sep='\t')
#df = pd.read_csv('hurricane_harvey_final_data.tsv', sep = '\t')
#df = pd.read_csv('hurricane_irma_final_data.tsv', sep = '\t')
#df = pd.read_csv('hurricane_maria_final_data.tsv', sep = '\t')
#df = pd.read_csv('iraq_iran_earthquake_final_data.tsv', sep = '\t')
#df = pd.read_csv('california_wildfires_final_data.tsv', sep = '\t')
df = pd.read_csv('mexico_earthquake_final_data.tsv', sep = '\t')
df = df.sample(frac=1)

df.drop(columns = ['image_id', 'text_info_conf', 'image_info_conf', 'image_human_conf', 'image_path', 'image_human', 'image_damage', 'image_damage_conf', 'text_human'])

,tweet_id,text_info,image_info,text_human_conf,tweet_text,image_url
0,910523364154003456,not_informative,not_informative,NaN,RT @FCBarcelona: Our solidarity with the victi...,http://pbs.twimg.com/media/DKLLdbyW0AASG2R.jpg
1,910523388598280192,informative,informative,1.0000,Mexico earthquake: Many children killed at pri...,http://pbs.twimg.com/media/DKLUMbJV4AACuFU.jpg
2,910523397448314882,not_informative,not_informative,NaN,Obamaâ€™s Response To The Earthquake In #Mexic...,http://pbs.twimg.com/media/DKLUND8WAAAHgON.jpg
3,910523398144397312,informative,informative,0.9282,RT @AmichaiStein1: #BREAKING: Israel search &a...,http://pbs.twimg.com/media/DKK1CuDWsAAX5Vk.jpg
4,910523398144397312,informative,informative,0.9282,RT @AmichaiStein1: #BREAKING: Israel search &a...,http://pbs.twimg.com/media/DKK1DyDXkAABvBX.jpg
...,...,...,...,...,...,...
1377,916105333872918528,informative,not_informative,1.0000,Carlos santana donates $100k to mexico earthqu...,http://pbs.twimg.com/media/DLao8g8W4AAKoPV.jpg
1378,916105374134042624,informative,not_informative,1.0000,Seth troxler donates 10k to earthquake relief ...,http://pbs.twimg.com/media/DLao-2GWsAA97vC.jpg
1379,916112796194021376,informative,informative,1.0000,Entercom/San Francisco Stations Raise Funds Fo...,http://pbs.twimg.com/media/DLavu1YV4AAgSJ_.jpg
1380,916164076484653056,informative,informative,0.6580,Mexico Earthquakes | International Medical Cor...,http://pbs.twimg.com/media/DLbeXxlWsAEl9xu.jpg


Filter the dataset so that it only contains those data points where text and image both are informative or non-informative. This done to ensure that there is only one label for the entire tweet. 

In [ ]:
# filtering the dataframe for the required fields


cond1 = df['text_info'] == 'informative'
cond2 = df['image_info'] == 'informative'
cond3 = df['text_info'] == 'not_informative'
cond4 = df['image_info'] == 'not_informative'

df_clean = df[(cond1&cond2) | (cond3&cond4)]
df_clean
print("Original Dataframe", len(df.index))
print("Filtered Dataframe", len(df_clean.index))

df_clean = df_clean[:1000]
df_short = df_clean.reset_index(drop = True)


Create the images directory, this will be used to store the locally downloaded tweet images 

In [ ]:
# create images directory incase it does not exist

!mkdir images

mkdir: cannot create directory ‘images’: File exists


Before starting the experiments for a new disaster dataset, be sure to remove the existing /images directory. Remove using this cell and create a new directory using the previous cell.

In [ ]:
!rm -rf images

Check if the indexes are properly orderd and reset them if required.

In [ ]:
# reset all the indexes

df_short
df_short = df_short.reset_index(drop = True)

##Generating Image Captions

Fetch the images from the URL given in the dataset and store it in /images folder with indexing

In [ ]:
# fetch the images from the URL's and store it in the /images folder 

import os
fail_count=0

for index, row in df_short.iterrows():
  curr_url = row['image_url']
  print(index)
  location = os.getcwd()+ '/images/' + str(index) + '.jpg'
  try:
    urllib.request.urlretrieve(curr_url, location)
    #print(curr_url)
  except:
    print("Image could not be downloaded")
    fail_count+=1

print(fail_count)

Upload the caption_multiply.py file from this https://github.com/codezakh/exploiting-BERT-thru-translation and move it to the catr folder

We need to to do this in order to ensure that we can generate captions for images in /images folder

In [ ]:
## upload caption_multiple.py from this repo: https://github.com/codezakh/exploiting-BERT-thru-translation
## and move it to catr folder

!mv caption_multiple.py catr/

Start generating the caption for images in the folder. It writes the caption for all the files in the json format 

In [ ]:
## run the caption_multiple.py file to generate captions for images in /images folder

!python catr/caption_multiple.py --path images/

Using cache found in /root/.cache/torch/hub/saahiluppal_catr_master
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0% 0/246 [00:00<?, ?it/s]/root/.cache/torch/hub/saahiluppal_catr_master/models/position_encoding.py:38: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (lik

Sample test run to verify if the captions are being produced. The text at the bottom of the cell is the caption produced for an image. 

In [ ]:
## sample test run to predict the caption generated for an image 

!python catr/predict.py --path e10.jpg

Using cache found in /root/.cache/torch/hub/saahiluppal_catr_master
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/content/catr/models/position_encoding.py:38: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorre

Create a filler in the form of "Caption" for the dataframe and fill it with some default string.

In [ ]:
#df_clean['Caption'] = 'No'
#print(df_clean.columns)
df_short['Caption'] = 'No'
df_short

Read the json file and map the caption to the index of the tweet in the dataframe 

In [ ]:
import json
f = open('images.json')
data = json.load(f)
type(data)
for key, value in data.items():
  index_value = key
  index_value_split = index_value.split(".")
  index_number = int(index_value_split[0])
  print(type(index_number))
  #df_clean.at[index_number, 'Caption'] = value
  df_short.at[index_number, 'Caption'] = value

In [ ]:
df_short


Clean the dataframe again and reset the indexes, if required

In [ ]:
#df_clean = df_clean[df_clean.Caption != 'No']
#df_clean
#df_clean.reset_index(drop = True)

df_short = df_short[df_short.Caption != 'No']
df_short

df_short = df_short.reset_index(drop = True)

In [ ]:
df_short

##Creation of Train-Test-Validation Splits

Here, we are just extracting the required columns for the classification task and creating the train, validation and test split. 

In [ ]:
df_class = df_short[['tweet_id', 'text_info', 'tweet_text', 'Caption']]

#df_class.text_info.replace(to_replace=["not_informative", "informative"], value=['0', '1'])
df_class["text_info"] = pd.Categorical(df_class["text_info"]).codes
df_class = df_class.rename(columns = {'text_info': 'label', 'tweet_text':'sentence1', 'Caption': 'sentence2'})

df_class

Create the train, test and validation splits for the sentence classification task using BERT

In [ ]:
# splitting into ratio of train-test and validation
print(len(df_class.index))
len_1 = int(0.7 * len(df_class.index))
len_2 = int(0.1 * len(df_class.index))


df_class_train = df_class[0:len_1]
df_class_val = df_class[len_1: len_1 + len_2]
df_class_test = df_class[len_1 +  len_2: len(df_class.index)]


df_class_train.to_csv('train.csv', index = False)
df_class_val.to_csv('val.csv', index = False)
df_class_test.to_csv('test.csv', index = False)

Once the csv files have been generated for each of the splits, remove the /images folder 

In [ ]:
!rm -rf images